<a href="https://colab.research.google.com/github/SwelteringCity/21052643_AI/blob/main/Genetic_Algorithm(Assignment2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Input

## Importing the packages

In [ ]:
import tensorflow as tf
from PIL import Image,ImageDraw
import random
import numpy as np
from skimage.metrics import structural_similarity as ssim
import cv2

## Image Path setting and defining the target_pixels

In [ ]:
image_path = 'https://github.com/SwelteringCity/21052643_AI/blob/main/Task_1/images/inputimage.png'
image = Image.open(image_path)
image.show()
canvas_width, canvas_height=image.size #canvas creation
target_image = np.array(image) #pixel array creation using numpy

Generating N random squares with random positions, RGB values, and opacity

In [6]:
# class Square:
#     def __init__(self):
#         # Random position within the canvas
#         self.x1 = random.randint(0, canvas_width)
#         self.y1 = random.randint(0, canvas_height)
#         self.x2 = random.randint(self.x1, canvas_width)
#         self.y2 = random.randint(self.y1, canvas_height)

#         # Random RGB color
#         self.color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

#         # Random opacity (0 to 255, where 0 is fully transparent and 255 is fully opaque)
#         self.opacity = random.randint(0, 255)

# # Create a population of random squares
# population = [Square() for _ in range(N)]


# Canvas generation

In [ ]:
#canvas_width, canvas_height = 800, 600
N = 10000  # Number of squares

# Create an empty canvas
canvas = Image.new("RGBA", (canvas_width, canvas_height), (255, 255, 255, 0))
draw = ImageDraw.Draw(canvas)

# Crossover

In [ ]:
# def custom_crossover(parent1, parent2):
#     # Create empty canvases for the offspring
#     offspring1 = create_empty_canvas()
#     offspring2 = create_empty_canvas()

#     # Randomly select a crossover point along the canvas
#     crossover_point = random_position()

#     # Copy elements from parent1 to offspring1 up to the crossover point
#     copy_elements(parent1, offspring1, up_to=crossover_point)

#     # Copy elements from parent2 to offspring2 from the crossover point
#     copy_elements(parent2, offspring2, from_position=crossover_point)

#     return offspring1, offspring2
def crossover(parent1, parent2):
    # Ensure parent images have the same dimensions
    if parent1.shape != parent2.shape:
        raise ValueError("Parent images must have the same dimensions for crossover.")

    # Create empty pixel arrays for the offspring
    height, width, channels = parent1.shape
    offspring1 = np.zeros((height, width, channels), dtype=np.uint8)
    offspring2 = np.zeros((height, width, channels), dtype=np.uint8)

    # Randomly select a crossover point along the width and height
    crossover_x = random.randint(0, width)
    crossover_y = random.randint(0, height)

    # Copy pixel values from parent1 to offspring1 up to the crossover point
    offspring1[:crossover_y, :crossover_x] = parent1[:crossover_y, :crossover_x]

    # Copy pixel values from parent2 to offspring2 from the crossover point
    offspring2[:crossover_y, :crossover_x] = parent2[:crossover_y, :crossover_x]

    return offspring1, offspring2


# Mutation

In [ ]:
# def mutate(self, mutation_rate, canvas_width, canvas_height):
#         # Mutate the square's attributes with a certain probability (mutation_rate)
#         if random.random() < mutation_rate:
#             # Mutate the x-coordinate
#             self.x = random.randint(0, canvas_width)

#         if random.random() < mutation_rate:
#             # Mutate the y-coordinate
#             self.y = random.randint(0, canvas_height)

#         if random.random() < mutation_rate:
#             # Mutate the size of the square
#             self.size = random.randint(1, 50)  # Adjust the size range as needed

#         if random.random() < mutation_rate:
#             # Mutate the color (assuming color is represented as an RGB tuple)
#             self.color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
import random
import numpy as np

class Square:
    def __init__(self, x, y, size, color):
        self.x = x
        self.y = y
        self.size = size
        self.color = color

    def mutate(self, mutation_rate, image_shape):
        # Mutate the square's attributes with a certain probability (mutation_rate)
        if random.random() < mutation_rate:
            # Mutate the x-coordinate
            self.x = max(0, min(self.x + random.randint(-10, 10), image_shape[1] - 1))

        if random.random() < mutation_rate:
            # Mutate the y-coordinate
            self.y = max(0, min(self.y + random.randint(-10, 10), image_shape[0] - 1))

        if random.random() < mutation_rate:
            # Mutate the size of the square
            self.size = max(1, min(self.size + random.randint(-10, 10), 50))  # Adjust the size range as needed

        if random.random() < mutation_rate:
            # Mutate the color (assuming color is represented as an RGB tuple)
            self.color = (
                max(0, min(self.color[0] + random.randint(-50, 50), 255)),
                max(0, min(self.color[1] + random.randint(-50, 50), 255)),
                max(0, min(self.color[2] + random.randint(-50, 50), 255))
            )

# Example usage:
# Initialize a Square instance
square = Square(10, 20, 30, (255, 0, 0))
mutation_rate = 0.1  # Adjust the mutation rate as needed
image_shape = (100, 100, 3)  # Adjust the image shape as needed

# Apply mutation to the square
square.mutate(mutation_rate, image_shape)

# Selection

In [ ]:
def roulette_wheel_selection(population, fitness_values):
    # Calculate the total fitness of the population
    total_fitness = sum(fitness_values)

    # Generate a random value between 0 and the total fitness
    random_value = random.uniform(0, total_fitness)

    # Initialize variables for selection
    cumulative_fitness = 0
    selected_square = None

    # Iterate through the population to select a square
    for i in range(len(population)):
        cumulative_fitness += fitness_values[i]
        if cumulative_fitness >= random_value:
            selected_square = population[i]
            break

    return selected_square

# Genetic Algorithm

## Objective function


In [ ]:
def objective_function(candidate_image):
    # Convert images to NumPy arrays (assuming they are in the same format)
    target_image = np.array(image)
    candidate_image_np = np.array(candidate_image)

    # Calculate SSIM
    ssim_value = ssim(target_image, candidate_image_np, multichannel=True)

    # SSIM returns a value in the range [-1, 1], where higher values indicate better similarity
    return 1 - ssim_value  # Convert SSIM to a minimization problem (lower values are better)

N=10000 #number of squares in the population
# Create an initial population of random squares
population = [Square() for _ in range(N)]

## Genetic Algorithm

In [ ]:
initial_mutation_rate = 0.2
mutation_reduction_factor = 0.95
max_generations = 100

# Initialize population with random squares

# Define objective function

# Genetic Algorithm loop
for generation in range(max_generations):
    # Calculate the mutation rate for the current generation
    mutation_rate = initial_mutation_rate / (generation + 1) * mutation_reduction_factor

    for square in population:
        # Calculate the objective function value for the square

        # Perform mutation with a variable mutation rate
        if random.random() < mutation_rate:
            # Apply mutation to the square

    # Evaluate fitness of mutated squares

    # Select the next generation based on fitness and other selection strategies

# The best individual in the final generation contains the squares to generate the image

# Final Output